In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

## **Load Data**

In [ ]:
train_data_loader = data.DataLoader() #TODO
test_data_loader = data.DataLoader() #TODO

## **Neural Network for Network Data**

In [5]:
class NetNet(nn.Module):
    def __init__(self, input_dim: int, latent_dim: int, output_dim: int):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, latent_dim)
        self.act = nn.ReLU()
        self.linear2 = nn.Linear(latent_dim, output_dim)

    def forward(self, x):
        x = self.linear1(x)
        x = self.act(x)
        x = self.linear2(x)
        return x
    
    def set_optim(self, learning_rate: float):
        self.optim = optim.Adam(self.parameters(), lr=learning_rate)

    def get_loss_fn(self):
        return nn.MSELoss()
    
    def train_model(self, epochs: int, data_loader: data.DataLoader, loss_fn: nn.MSELoss):
        self.train()

        for epoch in range(epochs):
            for inputs, labels in data_loader:
                outputs = self(inputs)
                loss = loss_fn(outputs, labels)
                loss.backward()
                self.optim.step()
                self.optim.zero_grad()

            if epoch % 10 == 1:
                print(f"Epoch: {epoch}, loss: {loss.item():.3}")
    
    def evaluate_model(self, data_loader: data.DataLoader):
        self.eval()
        true_predictions, predicitons_amount = 0., 0.

        with torch.no_grad():
            for inputs, labels in data_loader:
                predictions = self(inputs)
                predictions = predictions.squeeze(dim=1)
                true_predictions += (predictions == labels).sum()
                predicitons_amount += labels.shape[0]

            accuracy = 100.0 * true_predictions / predicitons_amount
        
        print(f"Accuracy of the model: {accuracy:4.2f}%")

In [6]:
input_dim = 64
latent_dim = 256
output_dim = 1
learning_rate = 0.001

In [7]:
model = NetNet(input_dim=input_dim, latent_dim=latent_dim, output_dim=output_dim)
model.set_optim(learning_rate=learning_rate)

### **Word2Vec**

In [8]:
import string
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
corpus = api.load('text8')
gensim_model = Word2Vec(corpus)

[==================================================] 100.0% 31.6/31.6MB downloaded


In [ ]:
class LabelsEmbeddings():
    def __init__(self, gensim_model: Word2Vec):
        self.model = gensim_model

    def generate_vectors(self, labels: list):
        vectors = []
        for label in labels:
            words = [word.translate(str.maketrans('', '', string.punctuation)) for word in label.split()]
            for word in words:
                if word in self.model.wv.vocab:
                    vectors.append(self.model.wv[word])
                else:
                    vectors.append([0])
        return vectors

### **Training** 

In [ ]:
loss_fn = model.get_loss_fn()
model.train_model(epochs=150, data_loader=train_data_loader, loss_fn=loss_fn)

In [ ]:
print(model)

### **Removing last layer**

In [ ]:
# TODO

In [ ]:
print(model)